In [1]:
import pandas as pd
import boxball_loader as bbl
import baseball_stats_utils as bsu

In [2]:
careers = bbl.load_batting(coalesce_type=bbl.CoalesceMode.PLAYER_CAREER)
careers

,g,ab,r,h,_2b,_3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
player_id,,,,,,,,,,,,,,,,,
aardsda01,331,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0,0.0
aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,73.0,1402,1383.0,293.0,32.0,21.0,121.0,328.0
aaronto01,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0,36.0
aasedo01,448,5,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0
abadan01,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zupofr01,16,18,3,3,1,0,0,0.0,0.0,0.0,2,6.0,0.0,0.0,0.0,0.0,0.0
zuvelpa01,209,491,41,109,17,2,2,20.0,2.0,0.0,34,50.0,1.0,2.0,18.0,0.0,8.0
zuverge01,266,142,5,21,2,1,0,7.0,0.0,1.0,9,39.0,0.0,0.0,16.0,0.0,3.0


In [3]:
ppl = bbl.load_people()
ppl.columns

Index(['player_id', 'birth_year', 'birth_month', 'birth_day', 'birth_country',
       'birth_state', 'birth_city', 'death_year', 'death_month', 'death_day',
       'death_country', 'death_state', 'death_city', 'name_first', 'name_last',
       'name_given', 'weight', 'height', 'bats', 'throws', 'debut',
       'final_game', 'retro_id', 'bbref_id'],
      dtype='object')

In [4]:
career_hr = pd.merge(left=careers[['hr']], right=ppl[['retro_id', 'player_id']], right_on='player_id', left_index=True).set_index('retro_id')['hr']
career_hr

retro_id
aardd001      0
aaroh101    755
aarot101     13
aased001      0
abada001      0
           ... 
zupof101      0
zuvep001      2
zuveg101      0
zwild101     30
zycht001      0
Name: hr, Length: 19898, dtype: int16

In [5]:
gls = bbl.load_gamelog_starters(bbl.GameType.ALL)
gls.columns

Index(['player_id', 'game_id', 'date', 'yr', 'game_type', 'team', 'HA', 'pos',
       'batting_pos'],
      dtype='object')

In [6]:
gls_hr = pd.merge(left=gls, right=career_hr, right_index=True, left_on='player_id')

In [7]:
lineup_hrs = gls_hr.groupby(['game_id', 'team', 'yr'])['hr'].sum().sort_values()
lineup_hrs

game_id       team  yr  
BR1187205090  BR1   1872       0
BR1187205070  BR1   1872       0
BR1187205180  BR1   1872       0
WS4187205170  WS3   1872       2
WS3187204180  WS3   1872       2
                            ... 
ALS199907130  NLS   1999    3537
ALS200307150  NLS   2003    3539
NLS200407130  NLS   2004    3659
ALS199907130  ALS   1999    3677
ALS200107100  NLS   2001    3935
Name: hr, Length: 405904, dtype: int16

In [8]:
lineup_hrs.tail(50)

game_id       team  yr  
MIN200604160  NYA   2006    3211
ANA200604070  NYA   2006    3211
OAK200604050  NYA   2006    3211
OAK200604040  NYA   2006    3211
OAK200604030  NYA   2006    3211
MIN200604140  NYA   2006    3211
TEX200605050  NYA   2006    3211
NYA200604270  NYA   2006    3211
NYA200604290  NYA   2006    3211
NYA200609290  NYA   2006    3212
NYA200610050  NYA   2006    3212
NYA200610030  NYA   2006    3212
NYA200609270  NYA   2006    3212
NYA201206080  NYA   2012    3218
NYA201205020  NYA   2012    3233
NLS200407130  ALS   2004    3241
NYA200506250  NYA   2005    3282
NYA200506150  NYA   2005    3282
BAL200509290  NYA   2005    3282
BAL200509280  NYA   2005    3282
OAK200509020  NYA   2005    3282
BOS200509300  NYA   2005    3282
NYA200506230  NYA   2005    3282
TOR200509170  NYA   2005    3282
KCA200506010  NYA   2005    3282
TOR200509180  NYA   2005    3282
TBA200509140  NYA   2005    3282
TEX200507180  NYA   2005    3282
NLS200207090  NLS   2002    3291
CLE200508030  NYA 

In [9]:
lineup_hrs.reset_index().groupby(['team', 'yr', 'hr'])['game_id'].count().reset_index().sort_values(by='hr').tail(50)

,team,yr,hr,game_id
55,ALS,1985,3016,1
129291,NYA,2005,3017,10
117954,NLS,1995,3028,1
129292,NYA,2005,3031,2
129293,NYA,2005,3032,2
129876,NYA,2012,3042,1
129792,NYA,2011,3042,4
129877,NYA,2012,3044,21
117956,NLS,1997,3046,1
129294,NYA,2005,3056,1


In [10]:
lineup_hrs.loc[lineup_hrs>3000].reset_index().groupby(['team'])['game_id'].count()

team
ALS      9
NLS     17
NYA    148
Name: game_id, dtype: int64

In [11]:
top_hr = lineup_hrs.tail(50).reset_index()
top_hr['url'] = top_hr['game_id'].apply(lambda g: bsu.create_bbref_boxscore_url(g))
top_hr

,game_id,team,yr,hr,url
0,MIN200604160,NYA,2006,3211,https://www.baseball-reference.com/boxes/MIN/M...
1,ANA200604070,NYA,2006,3211,https://www.baseball-reference.com/boxes/ANA/A...
2,OAK200604050,NYA,2006,3211,https://www.baseball-reference.com/boxes/OAK/O...
3,OAK200604040,NYA,2006,3211,https://www.baseball-reference.com/boxes/OAK/O...
4,OAK200604030,NYA,2006,3211,https://www.baseball-reference.com/boxes/OAK/O...
5,MIN200604140,NYA,2006,3211,https://www.baseball-reference.com/boxes/MIN/M...
6,TEX200605050,NYA,2006,3211,https://www.baseball-reference.com/boxes/TEX/T...
7,NYA200604270,NYA,2006,3211,https://www.baseball-reference.com/boxes/NYA/N...
8,NYA200604290,NYA,2006,3211,https://www.baseball-reference.com/boxes/NYA/N...
9,NYA200609290,NYA,2006,3212,https://www.baseball-reference.com/boxes/NYA/N...


In [12]:
top_hr['game_id'].apply(lambda g: bsu.create_bbref_boxscore_url(g)).apply(print)
None

https://www.baseball-reference.com/boxes/MIN/MIN200604160.shtml
https://www.baseball-reference.com/boxes/ANA/ANA200604070.shtml
https://www.baseball-reference.com/boxes/OAK/OAK200604050.shtml
https://www.baseball-reference.com/boxes/OAK/OAK200604040.shtml
https://www.baseball-reference.com/boxes/OAK/OAK200604030.shtml
https://www.baseball-reference.com/boxes/MIN/MIN200604140.shtml
https://www.baseball-reference.com/boxes/TEX/TEX200605050.shtml
https://www.baseball-reference.com/boxes/NYA/NYA200604270.shtml
https://www.baseball-reference.com/boxes/NYA/NYA200604290.shtml
https://www.baseball-reference.com/boxes/NYA/NYA200609290.shtml
https://www.baseball-reference.com/boxes/NYA/NYA200610050.shtml
https://www.baseball-reference.com/boxes/NYA/NYA200610030.shtml
https://www.baseball-reference.com/boxes/NYA/NYA200609270.shtml
https://www.baseball-reference.com/boxes/NYA/NYA201206080.shtml
https://www.baseball-reference.com/boxes/NYA/NYA201205020.shtml
https://www.baseball-reference.com/boxes

In [13]:
def get_lineups(game_id):
    df = gls_hr.query('game_id == @game_id').sort_values(['team', 'batting_pos'])
    df['name'] = bsu.get_player_names_col(df['player_id'], 'retro_id')
    return df[['team', 'batting_pos', 'name', 'hr']].rename(columns={'batting_pos': 'n'}).to_markdown(index=False)

print(get_lineups('ALS200107100'))

| team   |   n | name           |   hr |
|:-------|----:|:---------------|-----:|
| ALS    |   1 | Ichiro Suzuki  |  117 |
| ALS    |   2 | Alex Rodriguez |  696 |
| ALS    |   3 | Manny Ramirez  |  555 |
| ALS    |   4 | Bret Boone     |  252 |
| ALS    |   5 | Juan Gonzalez  |  434 |
| ALS    |   6 | John Olerud    |  255 |
| ALS    |   7 | Edgar Martinez |  309 |
| ALS    |   8 | Cal Ripken     |  431 |
| ALS    |   9 | Ivan Rodriguez |  311 |
| NLS    |   1 | Luis Gonzalez  |  354 |
| NLS    |   2 | Todd Helton    |  369 |
| NLS    |   3 | Barry Bonds    |  762 |
| NLS    |   4 | Sammy Sosa     |  609 |
| NLS    |   5 | Larry Walker   |  383 |
| NLS    |   6 | Mike Piazza    |  427 |
| NLS    |   7 | Chipper Jones  |  468 |
| NLS    |   8 | Jeff Kent      |  377 |
| NLS    |   9 | Rich Aurilia   |  186 |
